In [ ]:
#--------Loading in Packages------#
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from scipy.io import mmread
from scipy.sparse import csr_matrix
import seaborn as sns
import os
import tacco as tc

In [ ]:
#------Loading in 8um bin filtered dataset-------#
b2c_adata = sc.read_h5ad('/projects/b1217/HHA/EL_S2_B2C/anndatas/B2C_HE_Segmentation_NoDestripe_Cell_adata_11_13_25.h5ad')
b2c_adata

In [ ]:
#--------Computing QC Metrics for Spatial-----------#
sc.pp.calculate_qc_metrics(b2c_adata, inplace=True, log1p=True)
b2c_adata.obs

In [ ]:
#----------Plotting QC---------#
sc.pl.violin(b2c_adata,
        ["total_counts", "n_genes_by_counts", "bin_count"],
        multi_panel = True,
        stripplot=False)

In [ ]:
#-----------Adding Filter for Low Count Spots-----------#
b2c_adata.obs["PassedFilter"] = (b2c_adata.obs["total_counts"] > 50) & (b2c_adata.obs["n_genes_by_counts"] > 50)
b2c_adata.obs["PassedFilter"].value_counts()

In [ ]:
#----------Plotting Filtering on Spatial---------#
sc.set_figure_params(fontsize=20,figsize=[10,10])
sc.pl.spatial(b2c_adata, color=["PassedFilter"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer", s=10)

In [ ]:
#--------Subsetting for Spots with Good Counts--------#
b2c_adata = b2c_adata[b2c_adata.obs["PassedFilter"] == True].copy()
b2c_adata

In [ ]:
#----------Plotting Filtered Counts---------#
sc.set_figure_params(fontsize=20,figsize=[10,10])
sc.pl.spatial(b2c_adata, color=["PassedFilter"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer", s=10)

In [ ]:
#----------Saving Filtered Object---------#
b2c_adata.write_h5ad('/projects/b1217/HHA/EL_S2_B2C/anndatas/B2C_HE_Segmentation_NoDestripe_Cell_adata_Filtered_11_13_25.h5ad')

In [ ]:
#--------Loading in Full scRNA Atlas-----#
HHA = sc.read_h5ad("/projects/b1217/Edward/R_Projects/HHA/h5ad/20250516_HHA-SC-Final.h5ad")
HHA

In [ ]:
#--------Plotting UMAPs------#
#General Annotation
sc.pl.umap(HHA, color="GeneralAnnotation", frameon=False, legend_loc="on data")
#Final Annotation
sc.pl.umap(HHA, color="FinalAnnotation", frameon=False, legend_loc="on data")

In [ ]:
#-------Adding Gene Symbol Column to HHA----------#
#Same as names
HHA.var["symbol"] = HHA.var["names"].copy()
HHA.var

In [ ]:
#-------Formatting Var Names for Spatial----------#
#Same as names
b2c_adata.var["symbol"] = b2c_adata.var_names.copy()
b2c_adata.var["ensembl"] = b2c_adata.var["gene_ids"].copy()
b2c_adata.var_names = b2c_adata.var["ensembl"]
b2c_adata.var

In [ ]:
#------Printing Cell Counts for each Annotation------#
for cat, n in HHA.obs["FinalAnnotation"].value_counts().items():
    print(f"{cat}: {n}")

In [ ]:
#-------Converting gene expression matrices to float for TACCO------#
HHA.X = HHA.X.astype(np.float64)
b2c_adata.X = b2c_adata.X.astype(np.float64)

In [ ]:
#------Running TACCO on Full Dataset-------#
tc.tl.annotate(b2c_adata, HHA, annotation_key='FinalAnnotation', result_key='FinalAnnotation')

In [ ]:
#-----Retrieving Best Annotation Mapping for Each Cluster------#
tc.utils.get_maximum_annotation(b2c_adata, 'FinalAnnotation', result_key='FinalAnnotation')
b2c_adata

In [ ]:
#-------Plotting Global TACCO Results-----#
#Pulling Palette from scRNA adata
FinalAnnotation_palette = dict(zip(HHA.obs["FinalAnnotation"].cat.categories, HHA.uns["FinalAnnotation_colors"]))
FinalAnnotation_palette

#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(b2c_adata, color=["FinalAnnotation"], img_key="hires", ax=ax, palette = FinalAnnotation_palette)

In [ ]:
#---------Plotting Bulb Zoom----------#
#show masking based on cell
cell_mask = ((b2c_adata.obs['array_row'] >= 2800) & 
             (b2c_adata.obs['array_row'] <= 3200) & 
             (b2c_adata.obs['array_col'] >= 2520) & 
             (b2c_adata.obs['array_col'] <= 3020))
bulb_zoom = b2c_adata[cell_mask]
sc.set_figure_params(fontsize=20,figsize=[7,7])
sc.pl.spatial(bulb_zoom, color=["FinalAnnotation"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer", s=10)

In [ ]:
#---------------Pulling Scores from Obsm----------------#
Annot_Scores = b2c_adata.obsm['FinalAnnotation'].copy()
Annot_Names = [x.replace(" ", "_") + "_Score" for x in Annot_Scores.columns]
Annot_Scores.columns = Annot_Names
Annot_Scores
#Adding to obs
for Score in Annot_Scores.columns:
    b2c_adata.obs[Score] = Annot_Scores[Score]
b2c_adata.obs

In [ ]:
b2c_adata.obs.columns

In [ ]:
#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(b2c_adata, color=["Melanocytes_I_Score"], img_key="hires", ax=ax, cmap = "turbo")
#fig.savefig("/projects/b1217/HHA/EL_S2_B2C/Melanocyte_I_Scores.png")

In [ ]:
#--------Saving Annotated Objects---------#
#Full Mapping
b2c_adata.write_h5ad('/projects/b1217/HHA/EL_S2_B2C/anndatas/B2C_HE_Segmentation_Filtered_Annotated_11_13_25.h5ad')